In [3]:
# Cargar el archivo parquet en un DataFrame llamado df_fleca

# Usamos pyarrow.parquet para poder acceder al schema y tipos de columnas del archivo parquet,
# lo que permite detectar y convertir tipos especiales como 'dbdate' antes de cargar a pandas.
import pandas as pd
import pyarrow.parquet as pq  

parquet_file = '../data/raw/raw_data_bq_forecasting_20250630.parquet'
table = pq.read_table(parquet_file)
schema = table.schema

# Identificar columnas con tipo 'dbdate'
dbdate_cols = [field.name for field in schema if str(field.type) == 'dbdate']

# Convertir columnas 'dbdate' a string en el objeto Table de pyarrow
for col in dbdate_cols:
    table = table.set_column(table.schema.get_field_index(col), col, table.column(col).cast('string'))

# Convertir a pandas desactivando la metadata de pandas
df_fleca = table.to_pandas(ignore_metadata=True)

df_fleca.head()

,fecha,hour,n_factura,zona_de_venta,producto,familia,cantidad,base_imponible,tipo_IVA,total,turno
0,2025-06-13,None,No facturado,SALON,CATALANA,PAN,1.0,1.45,10.0,1.6,None
1,2025-06-05,None,No facturado,SALON,CATALANA,PAN,1.0,1.45,10.0,1.6,None
2,2025-06-27,None,No facturado,SALON,CATALANA,PAN,1.0,1.45,10.0,1.6,None
3,2025-06-04,None,No facturado,SALON,CATALANA,PAN,1.0,1.45,10.0,1.6,None
4,2025-05-28,None,No facturado,SALON,CATALANA,PAN,1.0,1.45,10.0,1.6,None


In [4]:
# Nos quedamos solo con las columnas de interés
df_fleca = df_fleca[['fecha', 'familia', 'base_imponible']]
df_fleca.head()

,fecha,familia,base_imponible
0,2025-06-13,PAN,1.45
1,2025-06-05,PAN,1.45
2,2025-06-27,PAN,1.45
3,2025-06-04,PAN,1.45
4,2025-05-28,PAN,1.45


In [5]:
# Verificar el tipo de datos de las columnas
df_fleca.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187884 entries, 0 to 187883
Data columns (total 3 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   fecha           187884 non-null  object 
 1   familia         187876 non-null  object 
 2   base_imponible  187876 non-null  float64
dtypes: float64(1), object(2)
memory usage: 4.3+ MB


In [6]:
#Cambiar el tipo de dato de la columna 'fecha' a datetime
df_fleca['fecha'] = pd.to_datetime(df_fleca['fecha'], format='%Y-%m-%d', errors='coerce')
df_fleca.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187884 entries, 0 to 187883
Data columns (total 3 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   fecha           187884 non-null  datetime64[ns]
 1   familia         187876 non-null  object        
 2   base_imponible  187876 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 4.3+ MB


In [22]:
# Cambiar el tipo de dato de la columna 'familia' a string
df_fleca['familia'] = df_fleca['familia'].astype(str)
df_fleca.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187884 entries, 0 to 187883
Data columns (total 3 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   fecha           187884 non-null  datetime64[ns]
 1   familia         187884 non-null  object        
 2   base_imponible  187876 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 4.3+ MB


In [24]:
# Ver que el DataFrame tiene los rangos de fecha de datos y los tipos de datos adecuados
df_fleca.describe(include='all')

,fecha,familia,base_imponible
count,187884,187884,187876.000000
unique,NaN,22,NaN
top,NaN,CAFES,NaN
freq,NaN,76728,NaN
mean,2024-09-24 20:59:18.382831872,NaN,2.126538
min,2024-01-02 00:00:00,NaN,0.000000
25%,2024-05-25 00:00:00,NaN,1.360000
50%,2024-08-31 00:00:00,NaN,1.640000
75%,2025-02-05 00:00:00,NaN,2.360000
max,2025-06-30 00:00:00,NaN,287.270000


In [25]:
# Guardar el DataFrame transformado en la carpeta interim
df_fleca.to_parquet('../data/interim/validated_range_dates_20250630.parquet', index=False)